<a href="https://colab.research.google.com/github/immischein/ML-bandgap/blob/immanuel_code/Bandgaps_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing library and imports

In [ ]:
# You might need to reinstall the library manually, imports ans pip install:
!pip install mp_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.6/122.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:


# Import
from mp_api.client import MPRester
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

# Example code

In [41]:
with MPRester("VlQ1oWGAXlhWiKRkPOk5q1cP8dNS6mIu") as mpr:
  #Please edit features you want to filter from inside the search() command. for example:
  example_docs = mpr.materials.summary.search(band_gap=(.001, None))

Retrieving SummaryDoc documents:   0%|          | 0/81998 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/mp_api/client/core/client.py:834: UserWarning: Use the 'fields' argument to select only fields of interest to speed up data retrieval for large queries. Choose from: ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula

In [44]:
thisdict = {
    "lattice_constant_a": [doc.structure.lattice.a for doc in example_docs],
    "lattice_constant_b": [doc.structure.lattice.b for doc in example_docs],
    "lattice_constant_c": [doc.structure.lattice.c for doc in example_docs],
    "lattice_angles_alpha": [doc.structure.lattice.alpha for doc in example_docs],
    "lattice_angles_beta": [doc.structure.lattice.beta for doc in example_docs],
    "lattice_angles_gamma": [doc.structure.lattice.gamma for doc in example_docs],
    "lattice_volume": [doc.structure.lattice.volume for doc in example_docs],
    "mean_distance": [
        np.mean(doc.structure.distance_matrix[0])
        for doc in example_docs
    ],
    "max_distance": [
        np.max(doc.structure.distance_matrix[0])
        for doc in example_docs
    ],
    "min_distance": [
        np.min(doc.structure.distance_matrix[0])
        for doc in example_docs
    ],
    "std_distance": [
        np.std(doc.structure.distance_matrix[0])
        for doc in example_docs
    ],
    "spacegroup_number " : [doc.structure.get_space_group_info()[1] for doc in example_docs]
}

In [45]:
import pandas as pd

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(thisdict)

# Compute the correlation matrix
correlation_matrix = df.corr(numeric_only=True)

print(correlation_matrix)

                      lattice_constant_a  lattice_constant_b  \
lattice_constant_a              1.000000            0.488100   
lattice_constant_b              0.488100            1.000000   
lattice_constant_c              0.119228            0.162951   
lattice_angles_alpha           -0.061346            0.046754   
lattice_angles_beta             0.011344            0.032737   
lattice_angles_gamma           -0.019264           -0.056646   
lattice_volume                  0.604012            0.587553   
mean_distance                   0.379611            0.392355   
max_distance                    0.336429            0.333175   
min_distance                         NaN                 NaN   
std_distance                    0.179152            0.165103   
spacegroup_number              -0.026744           -0.064723   

                      lattice_constant_c  lattice_angles_alpha  \
lattice_constant_a              0.119228             -0.061346   
lattice_constant_b              0.1

In [46]:
import pickle

# Object to save (for example, your dictionary or DataFrame)
object_to_save = thisdict  # or use df if it's a pandas DataFrame

# File path to save the object
with open("data.pkl", "wb") as f:
    pickle.dump(object_to_save, f)

In [ ]:
aa = np.arange(10)

In [ ]:
aa

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
bb = np.random.shuffle(aa)

In [ ]:
bb

In [ ]:
list_final = []
for i in range(5):
    list_i = []
    for j in range(len(example_docs[i].structure.species)):
        list_i.append(example_docs[i].structure.species[j].electrical_resistivity)
        # print(list_i)
    listia = np.array(list_i)
    print(listia)
    # list_final.append([np.sum(listia), np.mean(listia), np.std(listia), np.max(listia), np.min(listia)])


[1.63e-08 1.63e-08 1.63e-08 1.63e-08 1.63e-08 1.63e-08 5.00e-04 5.00e-04
 5.00e-04 5.00e-04 5.00e-04 5.00e-04 5.00e-04 5.00e-04 5.00e-04 5.00e-04
 1.00e-07 1.00e-07 1.00e-07 1.00e-07 1.00e-07 1.00e-07 1.00e-07 1.00e-07
 1.00e-07 1.00e-07 1.00e-07 1.00e-07]
[1.63e-08 1.63e-08 1.63e-08 1.63e-08 1.63e-08 1.63e-08 1.15e-07 1.15e-07
 1.15e-07 1.15e-07 5.00e-04 5.00e-04 5.00e-04 5.00e-04 5.00e-04 5.00e-04
 1.00e-07 1.00e-07 1.00e-07 1.00e-07 1.00e-07 1.00e-07 1.00e-07 1.00e-07
 1.00e-07 1.00e-07 1.00e-07 1.00e-07]
[2.7e-08 2.7e-08 2.7e-08 2.7e-08 2.7e-08 2.7e-08 7e-08 7e-08 7e-08 7e-08
 2000000000000000.0 None None None None None None None None None None None
 None]
[2.7e-08 2.7e-08 2.7e-08 2.7e-08 2.7e-08 2.7e-08 7e-08 7e-08 7e-08 7e-08
 0.0001 None None None None None None None None None None None None]
[2.7e-08 2.7e-08 2.7e-08 2.7e-08 2.7e-08 2.7e-08 5.4e-08 5.4e-08 5.4e-08
 5.4e-08 5.4e-08 None None None None None None None None None None None
 None None None None None]


<ipython-input-56-9eab4cdb2338>:5: UserWarning: No data available for electrical_resistivity for O
  list_i.append(example_docs[i].structure.species[j].electrical_resistivity)
<ipython-input-56-9eab4cdb2338>:5: UserWarning: No data available for electrical_resistivity for N
  list_i.append(example_docs[i].structure.species[j].electrical_resistivity)


In [ ]:
 example_docs[0].structure.species[].electrical_resistivity

1.63e-08

In [ ]:
example_docs[4].structure.species[20].electrical_resistivity

<ipython-input-69-fb4b2de9d73f>:1: UserWarning: No data available for electrical_resistivity for N
  example_docs[4].structure.species[20].electrical_resistivity


In [ ]:
example_docs[4].structure.species

[Element Al,
 Element Al,
 Element Al,
 Element Al,
 Element Al,
 Element Al,
 Element W,
 Element W,
 Element W,
 Element W,
 Element W,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N,
 Element N]